# Character Recognition Model

In [4]:
import tensorflow as tf

In [5]:
import pandas as pd

In [36]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

Load in Data

In [102]:
def df_processing(input_set):
    features = input_set.copy()
    labels = features.pop(features.columns[0])
    
    #one hot non 0 feature values to 1
    #and make them floats
    feature_col_names = list(features.columns.values)
    features[feature_col_names] = features.where(features[feature_col_names] == 0, 1).astype('float32')
    
    output_tensor = tf.data.Dataset.from_tensor_slices((features, labels))
    
    return output_tensor

In [103]:
train_set = pd.read_csv('Data/emnist-letters-train.csv')

In [104]:
train_tensor = df_processing(train_set)

In [105]:
#check if the tensor worked
for feat, targ in train_tensor.take(1):
    print('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [106]:
#shuffle and batch
train_tensor_shuffle_batch = train_tensor.shuffle(len(train_set)).batch(15)

Model

In [107]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation = 'relu'),
        tf.keras.layers.Dense(50, activation = 'relu'),
        tf.keras.layers.Dense(27) #outputs [0, 27)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])
    
    return model

In [108]:
letter_model = get_model()

In [109]:
letter_model.fit(train_tensor_shuffle_batch, epochs = 10)

Epoch 1/10
5920/5920 [==============================] - 17s 2ms/step - loss: 0.8625 - sparse_categorical_accuracy: 0.7356
Epoch 2/10
5920/5920 [==============================] - 16s 2ms/step - loss: 0.5404 - sparse_categorical_accuracy: 0.8274
Epoch 3/10
5920/5920 [==============================] - 11s 2ms/step - loss: 0.4643 - sparse_categorical_accuracy: 0.8507
Epoch 4/10
5920/5920 [==============================] - 13s 2ms/step - loss: 0.4179 - sparse_categorical_accuracy: 0.8621
Epoch 5/10
5920/5920 [==============================] - 12s 2ms/step - loss: 0.3890 - sparse_categorical_accuracy: 0.8720
Epoch 6/10
5920/5920 [==============================] - 12s 2ms/step - loss: 0.3668 - sparse_categorical_accuracy: 0.8790
Epoch 7/10
5920/5920 [==============================] - 13s 2ms/step - loss: 0.3445 - sparse_categorical_accuracy: 0.8846
Epoch 8/10
5920/5920 [==============================] - 13s 2ms/step - loss: 0.3285 - sparse_categorical_accuracy: 0.8890
Epoch 9/10
5920/5920 [==

Save the Model

In [110]:
#save model as json
letter_model_json = letter_model.to_json()
with open("letter_model.json", "w") as json_file:
    json_file.write(letter_model_json)

#save weights as h5
letter_model.save_weights("letter_model.h5")